In [ ]:
# Interactive Price Prediction Charts - Matching Your Image Style
# This creates charts exactly like the one you showed

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Chart styling to match your image
CHART_STYLE = {
    'bg_color': '#1a1a1a',
    'grid_color': '#333333',
    'line_color': '#00ff88',
    'text_color': '#ffffff',
    'font_family': 'Arial, sans-serif'
}

def create_market_prediction_chart(data, predictions, title, y_range=None):
    """Create a chart that matches your image style exactly"""
    
    fig = go.Figure()
    
    # Add historical data
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data.values,
        mode='lines',
        name='Historical',
        line=dict(color=CHART_STYLE['line_color'], width=2),
        hovertemplate='%{y:.2f}<br>%{x}<extra></extra>'
    ))
    
    # Add predictions
    fig.add_trace(go.Scatter(
        x=predictions.index,
        y=predictions.values,
        mode='lines',
        name='Prediction',
        line=dict(color=CHART_STYLE['line_color'], width=2, dash='dot'),
        hovertemplate='%{y:.2f}<br>%{x}<extra></extra>'
    ))
    
    # Add the current point (like the green dot in your image)
    current_point = data.iloc[-1]
    fig.add_trace(go.Scatter(
        x=[data.index[-1]],
        y=[current_point],
        mode='markers',
        name='Current',
        marker=dict(color=CHART_STYLE['line_color'], size=8),
        hovertemplate=f'{current_point:.2f}<br>%{{x}}<extra></extra>'
    ))
    
    # Update layout to match your image
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(color=CHART_STYLE['text_color'], size=20),
            x=0.5
        ),
        plot_bgcolor=CHART_STYLE['bg_color'],
        paper_bgcolor=CHART_STYLE['bg_color'],
        font=dict(color=CHART_STYLE['text_color'], family=CHART_STYLE['font_family']),
        xaxis=dict(
            gridcolor=CHART_STYLE['grid_color'],
            zeroline=False,
            showgrid=True,
            title=dict(text='', font=dict(color=CHART_STYLE['text_color'])),
            tickfont=dict(color=CHART_STYLE['text_color']),
            rangeslider=dict(visible=False),
            type='date'
        ),
        yaxis=dict(
            gridcolor=CHART_STYLE['grid_color'],
            zeroline=False,
            showgrid=True,
            title=dict(text='', font=dict(color=CHART_STYLE['text_color'])),
            tickfont=dict(color=CHART_STYLE['text_color']),
            range=y_range
        ),
        showlegend=False,
        margin=dict(l=20, r=20, t=40, b=20),
        height=400
    )
    
    # Add time period buttons (like in your image)
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1D", step="day", stepmode="backward"),
                    dict(count=5, label="5D", step="day", stepmode="backward"),
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(count=5, label="5Y", step="year", stepmode="backward"),
                    dict(step="all", label="Max")
                ]),
                bgcolor=CHART_STYLE['bg_color'],
                bordercolor=CHART_STYLE['grid_color'],
                font=dict(color=CHART_STYLE['text_color']),
                activecolor=CHART_STYLE['line_color']
            ),
            rangeslider=dict(visible=False),
            type="date"
        )
    )
    
    return fig

# ============================================================================
# MAIN PREDICTION SYSTEM
# ============================================================================

def get_housing_market_data():
    """Get housing market data using Real Estate ETF as proxy"""
    print("🏠 Loading Housing Market Data...")
    
    # Use VNQ (Vanguard Real Estate ETF) as housing market proxy
    housing_etf = yf.download('VNQ', start='2020-01-01', end=datetime.now())['Adj Close']
    
    # Normalize to create housing index starting from 5000 (like your chart)
    base_value = 5000
    housing_index = (housing_etf / housing_etf.iloc[0]) * base_value
    
    return housing_index

def get_gas_price_data():
    """Get gas price data using energy ETF"""
    print("⛽ Loading Gas Price Data...")
    
    # Use USO (United States Oil Fund) as gas price proxy
    gas_etf = yf.download('USO', start='2020-01-01', end=datetime.now())['Adj Close']
    
    # Normalize to create gas price index starting from 3.00 (like typical gas prices)
    base_value = 3.00
    gas_index = (gas_etf / gas_etf.iloc[0]) * base_value
    
    return gas_index

def get_food_price_data():
    """Get food price data using Food/Agriculture ETF"""
    print("🍎 Loading Food Price Data...")
    
    # Use DBA (Invesco DB Agriculture Fund) as food price proxy
    food_etf = yf.download('DBA', start='2020-01-01', end=datetime.now())['Adj Close']
    
    # Normalize to create food price index starting from 100 (like CPI)
    base_value = 100
    food_index = (food_etf / food_etf.iloc[0]) * base_value
    
    return food_index

In [ ]:
def create_features(data):
    """Create advanced features for machine learning"""
    df = pd.DataFrame(data)
    df.columns = ['price']
    
    # Technical indicators
    df['ma_7'] = df['price'].rolling(window=7).mean()
    df['ma_30'] = df['price'].rolling(window=30).mean()
    df['ma_90'] = df['price'].rolling(window=90).mean()
    
    # Price momentum
    df['momentum_7'] = df['price'].pct_change(7)
    df['momentum_30'] = df['price'].pct_change(30)
    
    # Volatility
    df['volatility_7'] = df['price'].rolling(window=7).std()
    df['volatility_30'] = df['price'].rolling(window=30).std()
    
    # RSI (Relative Strength Index)
    def calculate_rsi(prices, periods=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    df['rsi'] = calculate_rsi(df['price'])
    
    # Price position (normalized)
    df['price_position'] = (df['price'] - df['price'].rolling(window=252).min()) / (df['price'].rolling(window=252).max() - df['price'].rolling(window=252).min())
    
    # Seasonal features
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['year'] = df.index.year
    
    # Cyclical encoding for seasonal features
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4)
    df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4)
    
    # Lag features
    for lag in [1, 2, 3, 7, 14, 30]:
        df[f'lag_{lag}'] = df['price'].shift(lag)
    
    # Drop NaN values
    df = df.dropna()
    
    return df

def train_prediction_model(data, model_type='rf'):
    """Train a machine learning model for price prediction"""
    print(f"🤖 Training {model_type.upper()} model...")
    
    # Create features
    df = create_features(data)
    
    # Prepare features and target
    feature_cols = [col for col in df.columns if col != 'price']
    X = df[feature_cols]
    y = df['price']
    
    # Scale features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train model
    if model_type == 'rf':
        model = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
    
    model.fit(X_scaled, y)
    
    return model, scaler, feature_cols

def predict_future_prices(data, model, scaler, feature_cols, days_ahead=730):
    """Predict future prices for the next 2 years (730 days)"""
    print(f"🔮 Predicting prices for the next {days_ahead} days...")
    
    # Start with the last available data point
    last_data = data.copy()
    predictions = []
    
    # Create date range for predictions
    last_date = last_data.index[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=days_ahead, freq='D')
    
    # Predict step by step
    for i in range(days_ahead):
        # Create features for current state
        df_current = create_features(last_data)
        
        if len(df_current) == 0:
            break
            
        # Get latest features
        X_current = df_current[feature_cols].iloc[-1:].values
        X_current_scaled = scaler.transform(X_current)
        
        # Make prediction
        pred = model.predict(X_current_scaled)[0]
        predictions.append(pred)
        
        # Add prediction to data for next iteration
        new_date = future_dates[i]
        last_data = pd.concat([last_data, pd.Series([pred], index=[new_date])])
        
        # Keep only recent data to prevent memory issues
        if len(last_data) > 1000:
            last_data = last_data.iloc[-500:]
    
    # Create prediction series
    prediction_series = pd.Series(predictions, index=future_dates[:len(predictions)])
    
    return prediction_series

def get_market_insights(data, predictions):
    """Generate market insights based on predictions"""
    current_price = data.iloc[-1]
    future_price = predictions.iloc[-1]
    
    change_percent = ((future_price - current_price) / current_price) * 100
    
    # Calculate volatility
    volatility = data.pct_change().std() * np.sqrt(252) * 100  # Annualized volatility
    
    # Trend analysis
    if change_percent > 10:
        trend = "📈 Strong Uptrend"
    elif change_percent > 0:
        trend = "📊 Moderate Uptrend"
    elif change_percent > -10:
        trend = "📉 Moderate Downtrend"
    else:
        trend = "📉 Strong Downtrend"
    
    insights = {
        'current_price': current_price,
        'predicted_price': future_price,
        'change_percent': change_percent,
        'volatility': volatility,
        'trend': trend
    }
    
    return insights

def create_comprehensive_dashboard():
    """Create the complete price prediction dashboard"""
    print("🚀 Creating Comprehensive Price Prediction Dashboard...")
    print("=" * 60)
    
    # Load all data
    housing_data = get_housing_market_data()
    gas_data = get_gas_price_data()
    food_data = get_food_price_data()
    
    # Train models
    housing_model, housing_scaler, housing_features = train_prediction_model(housing_data)
    gas_model, gas_scaler, gas_features = train_prediction_model(gas_data)
    food_model, food_scaler, food_features = train_prediction_model(food_data)
    
    # Generate predictions
    housing_predictions = predict_future_prices(housing_data, housing_model, housing_scaler, housing_features)
    gas_predictions = predict_future_prices(gas_data, gas_model, gas_scaler, gas_features)
    food_predictions = predict_future_prices(food_data, food_model, food_scaler, food_features)
    
    # Create charts
    housing_fig = create_market_prediction_chart(housing_data, housing_predictions, "🏠 Housing Market Index - 2 Year Prediction")
    gas_fig = create_market_prediction_chart(gas_data, gas_predictions, "⛽ Gas Price Index - 2 Year Prediction")
    food_fig = create_market_prediction_chart(food_data, food_predictions, "🍎 Food Price Index - 2 Year Prediction")
    
    # Generate insights
    housing_insights = get_market_insights(housing_data, housing_predictions)
    gas_insights = get_market_insights(gas_data, gas_predictions)
    food_insights = get_market_insights(food_data, food_predictions)
    
    # Display results
    print("\n📊 MARKET INSIGHTS SUMMARY")
    print("=" * 60)
    
    print(f"\n🏠 HOUSING MARKET:")
    print(f"Current Index: {housing_insights['current_price']:.2f}")
    print(f"2-Year Prediction: {housing_insights['predicted_price']:.2f}")
    print(f"Expected Change: {housing_insights['change_percent']:.1f}%")
    print(f"Volatility: {housing_insights['volatility']:.1f}%")
    print(f"Trend: {housing_insights['trend']}")
    
    print(f"\n⛽ GAS PRICES:")
    print(f"Current Index: {gas_insights['current_price']:.2f}")
    print(f"2-Year Prediction: {gas_insights['predicted_price']:.2f}")
    print(f"Expected Change: {gas_insights['change_percent']:.1f}%")
    print(f"Volatility: {gas_insights['volatility']:.1f}%")
    print(f"Trend: {gas_insights['trend']}")
    
    print(f"\n🍎 FOOD PRICES:")
    print(f"Current Index: {food_insights['current_price']:.2f}")
    print(f"2-Year Prediction: {food_insights['predicted_price']:.2f}")
    print(f"Expected Change: {food_insights['change_percent']:.1f}%")
    print(f"Volatility: {food_insights['volatility']:.1f}%")
    print(f"Trend: {food_insights['trend']}")
    
    # Show charts
    housing_fig.show()
    gas_fig.show()
    food_fig.show()
    
    return {
        'housing': {'data': housing_data, 'predictions': housing_predictions, 'insights': housing_insights},
        'gas': {'data': gas_data, 'predictions': gas_predictions, 'insights': gas_insights},
        'food': {'data': food_data, 'predictions': food_predictions, 'insights': food_insights}
    }

# ============================================================================
# ENHANCED PREDICTION SYSTEM WITH DEEP LEARNING
# ============================================================================

def create_lstm_model(sequence_length=30):
    """Create LSTM model for time series prediction"""
    try:
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import LSTM, Dense, Dropout
        from tensorflow.keras.optimizers import Adam
        
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)),
            Dropout(0.2),
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            Dense(25),
            Dense(1)
        ])
        
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model
    except ImportError:
        print("⚠️ TensorFlow not available. Using Random Forest instead.")
        return None

def prepare_lstm_data(data, sequence_length=30):
    """Prepare data for LSTM model"""
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))
    
    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])
    
    return np.array(X), np.array(y), scaler

def train_lstm_model(data, sequence_length=30, epochs=50):
    """Train LSTM model for price prediction"""
    print("🧠 Training LSTM Deep Learning Model...")
    
    lstm_model = create_lstm_model(sequence_length)
    if lstm_model is None:
        return None, None, None
    
    X, y, scaler = prepare_lstm_data(data, sequence_length)
    
    # Reshape for LSTM
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # Train model
    lstm_model.fit(X, y, epochs=epochs, batch_size=32, verbose=0)
    
    return lstm_model, scaler, sequence_length

def predict_with_lstm(data, lstm_model, scaler, sequence_length, days_ahead=730):
    """Predict future prices using LSTM"""
    if lstm_model is None:
        return None
    
    print("🔮 Generating LSTM predictions...")
    
    # Prepare last sequence
    scaled_data = scaler.transform(data.values.reshape(-1, 1))
    last_sequence = scaled_data[-sequence_length:]
    
    predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(days_ahead):
        # Reshape for prediction
        X_pred = current_sequence.reshape((1, sequence_length, 1))
        
        # Make prediction
        pred_scaled = lstm_model.predict(X_pred, verbose=0)[0, 0]
        predictions.append(pred_scaled)
        
        # Update sequence
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = pred_scaled
    
    # Inverse transform predictions
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    
    # Create date range
    last_date = data.index[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=days_ahead, freq='D')
    
    return pd.Series(predictions, index=future_dates)

def create_advanced_dashboard():
    """Create advanced dashboard with multiple models"""
    print("🚀 Creating Advanced AI Price Prediction Dashboard...")
    print("=" * 70)
    
    # Load data
    housing_data = get_housing_market_data()
    gas_data = get_gas_price_data()
    food_data = get_food_price_data()
    
    results = {}
    
    for name, data in [('Housing', housing_data), ('Gas', gas_data), ('Food', food_data)]:
        print(f"\n📊 Processing {name} Market...")
        
        # Train Random Forest model
        rf_model, rf_scaler, rf_features = train_prediction_model(data)
        rf_predictions = predict_future_prices(data, rf_model, rf_scaler, rf_features)
        
        # Train LSTM model (if available)
        lstm_model, lstm_scaler, seq_length = train_lstm_model(data)
        if lstm_model is not None:
            lstm_predictions = predict_with_lstm(data, lstm_model, lstm_scaler, seq_length)
        else:
            lstm_predictions = None
        
        # Create ensemble prediction (average of both models)
        if lstm_predictions is not None:
            ensemble_predictions = (rf_predictions + lstm_predictions) / 2
        else:
            ensemble_predictions = rf_predictions
        
        # Create chart
        fig = create_market_prediction_chart(data, ensemble_predictions, f"🎯 {name} Price Prediction - AI Ensemble Model")
        fig.show()
        
        # Store results
        results[name.lower()] = {
            'data': data,
            'rf_predictions': rf_predictions,
            'lstm_predictions': lstm_predictions,
            'ensemble_predictions': ensemble_predictions,
            'insights': get_market_insights(data, ensemble_predictions)
        }
    
    # Display summary
    print("\n🎯 ADVANCED AI PREDICTIONS SUMMARY")
    print("=" * 70)
    
    for name, result in results.items():
        insights = result['insights']
        print(f"\n{name.upper()} MARKET:")
        print(f"Current: {insights['current_price']:.2f}")
        print(f"AI Prediction (2 years): {insights['predicted_price']:.2f}")
        print(f"Expected Change: {insights['change_percent']:.1f}%")
        print(f"Trend: {insights['trend']}")
    
    return results

In [ ]:
# ============================================================================
# EXECUTE THE COMPLETE PREDICTION SYSTEM
# ============================================================================

# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    """Install package if not already installed"""
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
required_packages = [
    'yfinance', 'scikit-learn', 'plotly', 'pandas', 'numpy'
]

for package in required_packages:
    install_package(package)

print("✅ All required packages installed!")
print("🚀 Starting AI Price Prediction System...")
print("=" * 70)

# Run the comprehensive dashboard
results = create_comprehensive_dashboard()

print("\n🎯 PREDICTION SYSTEM COMPLETE!")
print("=" * 70)
print("📊 Interactive charts are displayed above")
print("💡 Charts include:")
print("   • 🏠 Housing Market 2-Year Prediction")
print("   • ⛽ Gas Price 2-Year Prediction") 
print("   • 🍎 Food Price 2-Year Prediction")
print("\n📈 Features:")
print("   • Real-time data from Yahoo Finance")
print("   • Advanced AI models (Random Forest)")
print("   • Interactive charts with time controls")
print("   • Market trend analysis")
print("   • Volatility calculations")
print("\n🔮 Next Steps:")
print("   • Run create_advanced_dashboard() for LSTM models")
print("   • Customize prediction periods")
print("   • Add more economic indicators")

print("\n✨ Ready for Google Colab! Copy this entire notebook.")

In [ ]:
# ============================================================================
# ADVANCED AI MODELS WITH DEEP LEARNING (OPTIONAL)
# ============================================================================

# Uncomment and run this cell for advanced LSTM predictions
# This requires TensorFlow which takes more time to install

"""
# Install TensorFlow for deep learning
!pip install tensorflow

# Run advanced dashboard with LSTM models
print("🧠 Running Advanced AI Dashboard with Deep Learning...")
advanced_results = create_advanced_dashboard()

print("\n🎯 ADVANCED AI PREDICTIONS COMPLETE!")
print("This includes both Random Forest and LSTM ensemble models")
"""

# ============================================================================
# CUSTOM PREDICTION FUNCTIONS
# ============================================================================

def predict_single_market(market_name, days_ahead=730):
    """Predict a single market for custom time period"""
    print(f"🔍 Analyzing {market_name} market...")
    
    if market_name.lower() == 'housing':
        data = get_housing_market_data()
    elif market_name.lower() == 'gas':
        data = get_gas_price_data()
    elif market_name.lower() == 'food':
        data = get_food_price_data()
    else:
        print("❌ Invalid market name. Use: 'housing', 'gas', or 'food'")
        return None
    
    # Train model and predict
    model, scaler, features = train_prediction_model(data)
    predictions = predict_future_prices(data, model, scaler, features, days_ahead)
    
    # Create chart
    fig = create_market_prediction_chart(data, predictions, f"📈 {market_name.title()} Price Prediction")
    fig.show()
    
    # Get insights
    insights = get_market_insights(data, predictions)
    
    print(f"\n📊 {market_name.upper()} ANALYSIS:")
    print(f"Current Price: {insights['current_price']:.2f}")
    print(f"Predicted Price ({days_ahead} days): {insights['predicted_price']:.2f}")
    print(f"Expected Change: {insights['change_percent']:.1f}%")
    print(f"Trend: {insights['trend']}")
    
    return {'data': data, 'predictions': predictions, 'insights': insights}

def export_predictions_to_csv(results, filename='price_predictions.csv'):
    """Export predictions to CSV file"""
    print(f"💾 Exporting predictions to {filename}...")
    
    export_data = []
    
    for market, result in results.items():
        predictions = result['predictions']
        for date, price in predictions.items():
            export_data.append({
                'Date': date,
                'Market': market.title(),
                'Predicted_Price': price,
                'Type': 'Prediction'
            })
    
    df = pd.DataFrame(export_data)
    df.to_csv(filename, index=False)
    print(f"✅ Predictions exported to {filename}")
    
    return df

# ============================================================================
# MARKET COMPARISON DASHBOARD
# ============================================================================

def create_comparison_dashboard():
    """Create a comparison dashboard showing all markets together"""
    print("📊 Creating Market Comparison Dashboard...")
    
    # Load all data
    housing_data = get_housing_market_data()
    gas_data = get_gas_price_data()
    food_data = get_food_price_data()
    
    # Normalize all data to same scale for comparison
    housing_norm = (housing_data / housing_data.iloc[0]) * 100
    gas_norm = (gas_data / gas_data.iloc[0]) * 100
    food_norm = (food_data / food_data.iloc[0]) * 100
    
    # Create comparison chart
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=housing_norm.index,
        y=housing_norm.values,
        mode='lines',
        name='Housing Market',
        line=dict(color='#ff6b6b', width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=gas_norm.index,
        y=gas_norm.values,
        mode='lines',
        name='Gas Prices',
        line=dict(color='#4ecdc4', width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=food_norm.index,
        y=food_norm.values,
        mode='lines',
        name='Food Prices',
        line=dict(color='#45b7d1', width=2)
    ))
    
    fig.update_layout(
        title="📈 Market Comparison - Normalized Performance",
        xaxis_title="Date",
        yaxis_title="Index (Base = 100)",
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white'),
        showlegend=True,
        height=500
    )
    
    fig.show()
    
    return fig

# Usage examples:
print("\n🎯 USAGE EXAMPLES:")
print("=" * 50)
print("1. Predict single market:")
print("   predict_single_market('housing', days_ahead=365)")
print("   predict_single_market('gas', days_ahead=180)")
print("   predict_single_market('food', days_ahead=730)")
print("\n2. Create comparison dashboard:")
print("   create_comparison_dashboard()")
print("\n3. Export predictions:")
print("   export_predictions_to_csv(results)")
print("\n4. Run advanced AI models:")
print("   # Uncomment the TensorFlow section above")
print("\n✨ Ready to use in Google Colab!")

In [1]:
# ============================================================================
# MARKET INSIGHTS BASED ON CURRENT TRENDS (July 2025)
# ============================================================================

def display_market_insights():
    """Display current market insights based on research"""
    print("🔍 CURRENT MARKET INSIGHTS & ANALYSIS")
    print("=" * 70)
    
    print("\n🏠 HOUSING MARKET INSIGHTS:")
    print("   • Housing affordability remains a major challenge")
    print("   • Supply constraints continue to affect prices")
    print("   • Regional variations significant (coastal vs inland)")
    print("   • Interest rates impact on mortgage demand")
    print("   • Prediction: Moderate price appreciation 2-5% annually")
    
    print("\n⛽ GAS PRICE INSIGHTS:")
    print("   • Current price around $3.34/MMBtu for natural gas")
    print("   • Geopolitical tensions affecting oil/gas markets")
    print("   • Seasonal demand patterns (winter heating)")
    print("   • Environmental policies driving energy transition")
    print("   • Prediction: Moderate volatility with gradual increase")
    
    print("\n🍎 FOOD PRICE INSIGHTS:")
    print("   • Food inflation has slowed but remains elevated")
    print("   • Some items still 50%+ higher than 2019 levels")
    print("   • Weather and climate affecting crop yields")
    print("   • Supply chain disruptions continuing")
    print("   • Prediction: Gradual normalization with occasional spikes")
    
    print("\n📊 KEY ECONOMIC FACTORS:")
    print("   • Federal Reserve policy on interest rates")
    print("   • Inflation targeting around 2%")
    print("   • Supply chain resilience improvements")
    print("   • Consumer spending patterns")
    print("   • Global economic stability")
    
    print("\n⚠️  PREDICTION DISCLAIMERS:")
    print("   • Predictions are based on historical patterns")
    print("   • Market conditions can change rapidly")
    print("   • External factors (policy, weather, etc.) impact prices")
    print("   • Use predictions as guidance, not financial advice")
    print("   • Consider multiple scenarios and risk management")

# ============================================================================
# GOOGLE COLAB SETUP INSTRUCTIONS
# ============================================================================

def colab_setup_instructions():
    """Display instructions for Google Colab setup"""
    print("\n🚀 GOOGLE COLAB SETUP INSTRUCTIONS")
    print("=" * 70)
    
    print("\n1. Copy this entire notebook to Google Colab")
    print("2. Run the first cell to install all required packages")
    print("3. Execute cells in order (each cell takes 1-2 minutes)")
    print("4. Interactive charts will appear below each cell")
    print("5. For advanced models, uncomment the TensorFlow section")
    
    print("\n📋 EXPECTED RUNTIME:")
    print("   • Basic predictions: 3-5 minutes")
    print("   • Advanced LSTM models: 10-15 minutes")
    print("   • Data loading: 1-2 minutes per market")
    
    print("\n🔧 CUSTOMIZATION OPTIONS:")
    print("   • Change prediction period (days_ahead parameter)")
    print("   • Modify chart styling (CHART_STYLE dictionary)")
    print("   • Add new markets (extend data loading functions)")
    print("   • Adjust model parameters (RandomForestRegressor)")
    
    print("\n💡 TROUBLESHOOTING:")
    print("   • If data loading fails, check internet connection")
    print("   • For TensorFlow issues, restart runtime")
    print("   • Charts not showing? Try fig.show() command")
    print("   • Memory issues? Reduce prediction days_ahead")

# ============================================================================
# FINAL EXECUTION
# ============================================================================

# Display insights and instructions
display_market_insights()
colab_setup_instructions()

print("\n🎯 SYSTEM READY!")
print("=" * 70)
print("✅ AI Price Prediction System Complete")
print("📊 Run create_comprehensive_dashboard() to start")
print("🚀 Perfect for Google Colab deployment")
print("📈 Predicts 2-year price trends for Housing, Gas, and Food")
print("🤖 Uses advanced machine learning models")
print("📱 Interactive charts with time controls")
print("💾 Export capabilities for further analysis")
print("\n🌟 FEATURES INCLUDED:")
print("   • Real-time data fetching")
print("   • Advanced feature engineering")
print("   • Multiple ML models (Random Forest, LSTM)")
print("   • Interactive visualizations")
print("   • Market trend analysis")
print("   • Volatility calculations")
print("   • Export to CSV")
print("   • Comparison dashboards")
print("\n✨ Happy Predicting! 🚀")

🔍 CURRENT MARKET INSIGHTS & ANALYSIS

🏠 HOUSING MARKET INSIGHTS:
   • Housing affordability remains a major challenge
   • Supply constraints continue to affect prices
   • Regional variations significant (coastal vs inland)
   • Interest rates impact on mortgage demand
   • Prediction: Moderate price appreciation 2-5% annually

⛽ GAS PRICE INSIGHTS:
   • Current price around $3.34/MMBtu for natural gas
   • Geopolitical tensions affecting oil/gas markets
   • Seasonal demand patterns (winter heating)
   • Environmental policies driving energy transition
   • Prediction: Moderate volatility with gradual increase

🍎 FOOD PRICE INSIGHTS:
   • Food inflation has slowed but remains elevated
   • Some items still 50%+ higher than 2019 levels
   • Weather and climate affecting crop yields
   • Supply chain disruptions continuing
   • Prediction: Gradual normalization with occasional spikes

📊 KEY ECONOMIC FACTORS:
   • Federal Reserve policy on interest rates
   • Inflation targeting around 